In [1]:
%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import sys
import cv2
import random
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import pretrainedmodels.utils as utils
import torch
import torch.optim as optim
from apex import amp
from easydict import EasyDict
from numpy.random import choice, seed
from PIL import Image
from pretrainedmodels import xception
from torch import nn
from torch import from_numpy
from torch.utils.data import Dataset, SequentialSampler
from torchvision import transforms as tt
from torchvision.transforms.functional import to_tensor
import tqdm
from tqdm import tqdm_notebook
tqdm.tqdm = tqdm_notebook
tqdm = tqdm_notebook
# from unet.unet_model import UNet
from segmentation_models_pytorch import Unet
from custom_pytorch import custom_layers
from custom_pytorch.custom_models import SamplingSegmentationV3 as SamplingSegmentation
from custom_pytorch.custom_samplers import SubsetRandomSampler
from custom_pytorch.custom_schedulers import CyclicLRWithRestarts
from custom_pytorch.custom_losses.dice import *
from custom_pytorch.metrics import DiceCoeff
from custom_pytorch.custom_utils import params_number, submodules_number
from custom_pytorch.custom_visualizations.segmentation import Visualizer
from custom_pytorch.custom_utils import get_model_name
from custom_pytorch.optimizers.adabound import AdaBound
from torchcontrib.optim import SWA

# dice_loss = GeneralizedDiceLoss()
import PyQt5
sys.path.insert(0, '../')
from shared import *
from config import CONFIG
import fastai

Training folder data size: 10675
Testing folder data size: 1377


In [4]:
from custom_pytorch.custom_models import SEXceptionXUnet

In [5]:
ENCODER = 'se_resnext50_32x4d'
ENCODER_WEIGHTS = 'imagenet'
DEVICE = 'cuda'

ACTIVATION = 'sigmoid'
# create segmentation model with pretrained encoder
# model = smp.Unet(
#     encoder_name=ENCODER, 
#     encoder_weights=ENCODER_WEIGHTS, 
#     classes=1, 
#     activation=ACTIVATION,
# )
# model.name = 'unet'
sample = get_dataset('train')[0]['image']
sample = sample.unsqueeze(dim=0)

from custom_pytorch.models.efficient_net_encoder import EfficientNetEncoder


from efficientunet import *

encoder = EfficientNetEncoder('efficientnet-b3', pretrained=True).cuda()
CONFIG.identifier='EfficientNet_XUnet'

# model = SamplingSegmentation(3, 1, 5, 2)

# model = SEXceptionXUnet(ENCODER, sample, 1, reversed_features=True).cuda()
model = SEXceptionXUnet(encoder, sample, 1, reversed_features=False).cuda()
# model = torch.load("best_EfficientNet_XUnet_D_2019-08-21_03:41:13.637327_Size_256_Ep_6_TL_6.86_VL_8.57_TM_0.24_VM_0.11.pth")
# model = SamplingSegmentation(3, 1, CONFIG.net_params.depth, CONFIG.net_params.resolution).cuda()
# net = NeuralNet().cuda()



# print("Total number of trainable parameters:", par_num(model))
# print("Total number of encoder parameters:", par_num(model.encoder))
# print("Total number of decoder parameters:", par_num(model.decoder))
# print("Total number of output layer parameters:", par_num(model.output_layer))
# for c, col in enumerate(model.decoder['decoding_columns']):
#     print('')
#     print('Column ', c)
#     print("Number of decoders: ", len(col.column_decoders))
#     print("Dimensions of decoders: ", [sub_num(dec) for dec in col.column_decoders])
#     print("Parameters of decoders: ", [par_num(dec) for dec in col.column_decoders])
#     print("Number of downsamplers: ", len(col.column_downsamplers))
#     print("Dimensions of downsamplers: ", [sub_num(dec) for dec in col.column_downsamplers])
#     print("Number of modules: ", sub_num(col))
#     column_parameters = filter(lambda p: p.requires_grad, col.parameters())
#     params = sum([np.prod(p.size()) for p in column_parameters])
#     print("Number of parameters: ", params)
# print('')
# print("Dimensions of output layer: ", sub_num(model.output_layer))

In [6]:
# decoding_column_ind = 1
# column_decoder_ind = 0
# {key: (value.size(), np.prod(value.size())) for key, value in model.decoder['decoding_columns'][decoding_column_ind].column_decoders[column_decoder_ind].named_parameters()}

In [7]:
# model.decoder['decoding_columns'][decoding_column_ind].column_decoders[column_decoder_ind].reps

In [8]:
from custom_pytorch.external.onecyclelr import OneCycleLR


def train_model():
    model.train()
    lr = CONFIG.lr
    momentum = CONFIG.momentum
    
    # scheduler = AdaptiveCyclicCosineAnnealing(CONFIG.lr,  optimizer, 200, CONFIG.train_batches_num, snapshots_num=8)


    train_losses = {}
    train_coeffs = {}
    valid_losses = {}
    valid_coeffs = {}
    step = 0
    run_validation_every_n_steps = 100

    examples_savedir = os.path.join(LOGS_SAVE_DIR, 'visual')
    try:
        os.makedirs(examples_savedir)
    except OSError:
        pass


    VISUAL_EXAMPLES_EVERY = 50
    TRAIN_PLOT_EVERY = 50
    partial_loss = 0
    partial_coeff = 0

    # visualizer = Visualizer(CONFIG, metric_used='Dice Coefficient', include_lr=True, examples_savedir=examples_savedir)

#     loss = CEDiceLoss(with_logits=True)
#     loss = WindowedCEDiceLoss()
#     loss = MultiWindowedBCELoss(with_logits=True, win_num=20)
#     loss = BCEAndMultiWindowedDiceLoss(with_logits=True, dice_loss_kwargs={'win_num': 10})
#     loss = MultiWindowedDiceLoss(with_logits=True, win_num=10)
    loss = BCEAndDiceLoss(with_logits=True)
#     loss = ExpandedBCEDiceLoss(with_logits=True, win_num=10)
#     loss = MultiWindowedDiceLoss(with_logits=True, win_num=10)
    
    metrics = [
            smp.utils.metrics.IoUMetric(eps=1.),
            smp.utils.metrics.FscoreMetric(eps=1.),
        ]
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=0.0001)
    early_epochs = 0
#   scheduler = OneCycleLR(optimizer, num_steps=early_epochs, lr_range=(lr, min(1, 10 * lr)))
    trainer = Trainer(model, optimizer, loss, metrics)
    trainer.load_model('EfficientNet_XUnet_D_2019-08-21_23:19:17.405013_Size_256_Ep_1_TL_1.99_VL_2.13_TM_0.02_VM_0.01best.pth')

    # train model for 40 epochs

    max_score = 0
    stale_epochs_num = 0
    min_loss = 10000
    ALLOWED_STALE = 2 * CONFIG.restart_period
    SWA_START = 4 * CONFIG.restart_period
    best_model_name = None
    while True:
        if trainer.epoch == early_epochs:
#             print("\n\nOne cycle policy finished, starting Cyclic LR schedule with lower LR\n\n")
#             trainer.optimizer = SWA(optimizer, 
#                                     swa_start=SWA_START * CONFIG.train_batches_number,
#                                     swa_freq=2 * CONFIG.train_batches_number,
#                                     swa_lr=0.1 * CONFIG.lr)
#             trainer.optimizer = torch.optim.Adam(model.parameters(), lr=CONFIG.lr)
#             scheduler = CyclicLRWithRestarts(trainer.optimizer, CONFIG.batch_size, CONFIG.train_size, restart_period=CONFIG.restart_period,
#                                           t_mult=1.2, min_lr= 1e-2 * CONFIG.lr)
            scheduler = optim.lr_scheduler.ReduceLROnPlateau(trainer.optimizer, mode='min', verbose=True)

        print('\nEpoch: {}, LR: {}'.format(trainer.epoch + 1, trainer.optimizer.param_groups[0]['lr']))
        train_logs = trainer.step(logs=trainer.train_loss_logs, valid=False)
        trainer.write_logs(train_logs, valid=False)
        valid_logs = trainer.step(logs=trainer.valid_loss_logs, valid=True)
        trainer.write_logs(valid_logs, valid=True)
        


        # do something (save model, change lr, etc.)
        stale_epoch = True
        if max_score < valid_logs['iou']:
            stale_epochs_num = 0
            max_score = valid_logs['iou']
            trainer.save_best_model(valid_metric=max_score, train_metric=train_logs['iou'], valid_loss=valid_logs[loss.__name__], train_loss=train_logs[loss.__name__])
            print('Model saved!')
            stale_epoch = False
        if valid_logs[loss.__name__] < min_loss:
            min_loss = valid_logs[loss.__name__]
            stale_epoch = False
            print("Minimum valid loss was found, the stale counter has reset.")
        if stale_epoch:
            stale_epochs_num += 1
        if stale_epochs_num == ALLOWED_STALE:
            print(f"No improvements for the last {ALLOWED_STALE} epochs, the training has finished")
            print(f"Best validation IoU score: {max_score}")
            break
        scheduler.step(train_logs[loss.__name__])
#         if i < SWA_START:
#             scheduler.step()
#     trainer.optimizer.swap_swa_sgd()
    trainer.save_last_model(valid_metric=max_score, train_metric=train_logs['iou'], valid_loss=valid_logs[loss.__name__], train_loss=train_logs[loss.__name__])
    return best_model_name

In [9]:
params_number(model)

12858504

In [10]:
submodules_number(model)

1115

In [11]:
best_model_name = train_model()

Number of high weight indices found:  2102
The following attributes of the configuration differ from the ones loaded:


,loaded,current
date,2019-08-21 23:19:17.405013,2019-08-21 23:26:51.697879
train_batches_number,5,500
train_size,20,2000



Epoch: 3, LR: 0.01


KeyboardInterrupt: 

In [ ]:
print("Training model using same LR for both encoder and decoder")
# model = torch.load("best_xUnet_encoder_D_2019-07-27_01:35:05.984560_Ep_129_TL_0.49_VL_0.48_TM_0.57_VM_0.57.pth")
model = torch.load(best_model_name)
best_model_name = train_model(mainly_encoder=False)

In [ ]:
model = torch.load(model)
best_model_name = train_model(mainly_encoder=False)

In [ ]:
model = torch.load(best_model_name)
best_model_name = train_model(mainly_encoder=False)

In [ ]:
best_model_name